# initial tests

In [14]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install pytest

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.3/325.3 kB 3.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [116]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

API_KEY=os.getenv("API_KEY")
MODEL="gpt-3.5-turbo-0125"

def listModels():
    url = "https://api.openai.com/v1/models"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + API_KEY,
    }
    response = requests.get(url, headers=headers)
    print(response.json())
def chatGPT(text, additional_context):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + API_KEY,
    }
    data = {
        "model": MODEL,
        "messages": [
          {
            "role": "system",
            "content": "You are a tool used to automatically generate source code for unit tests. Source code for a function that is to be tested will be provided and you should generate an appropriate unit test that can be ran to ensure the code is correct. Code input will be in Python and you should respond with a Python code block containing the unit test as output. Use Pytest. Do not start output with \"```python\". Don't use Parametrize. For numeric equality assertions, allow a tolerance of 1e^-6. Some additional context about the method under test is \""+additional_context+"\""
          },
          {
            "role": "user",
            "content": """
              def add(a, b):
                return a + b"
              """
          },
          {
            "role": "assistant",
            "content": "import pytest\n\ndef add(a, b):\n    return a + b\n\ndef test_add():\n    assert add(3, 5) == 8\n    assert add(-3, -5) == -8\n    assert add(3, -5) == -2\ntest_add()"
          },
          {
            "role": "user",
            "content": text
          },
        ],
        "max_tokens": 500,
        "temperature": 1.0,
    }
    response = requests.post(url, headers=headers, json=data)
#     print(response.json())
    output = response.json()['choices'][0]['message']['content']

    return output



In [127]:
import json
SAMPLES_TO_RUN = 5
out = []
count = 0
with open('HumanEval.jsonl') as file:
    for line in file:
        count = count + 1
        d = json.loads(line)
        d['soln'] = chatGPT(d['canonical_solution'], d["prompt"])

        check_program = (
                d["soln"] + "\n"
            )
        try:
            exec(check_program)
            out.append(f"{d['task_id']}, passes")
        except AssertionError as ae:
            print("Failing Test\n\n")
            print(check_program)
            out.append(f"{d['task_id']}, failed assertion")
        except Exception as e:
            out.append(f"{d['task_id']}, failed to run")
            print(check_program)
        
        if count > SAMPLES_TO_RUN-1:
            break
print(out)

['HumanEval/0, passes', 'HumanEval/1, passes', 'HumanEval/2, passes', 'HumanEval/3, passes', 'HumanEval/4, passes']


In [126]:
def parse_paren_group(s):
    depth = 0
    max_depth = 0
    for c in s:
        if c == '(':
            depth += 1
            max_depth = max(depth, max_depth)
        else:
            depth -= 1

    return max_depth

def parse_nested_parens(paren_string):
    return [parse_paren_group(x) for x in paren_string.split(' ') if x]

def test_parse_nested_parens():
    assert parse_nested_parens('(()()) ((())) () ((())()())') == [2, 3, 1, 3]
    assert parse_nested_parens(')(())') == [1]
    assert parse_nested_parens('') == []
    assert parse_nested_parens('(()) (())') == [2, 2]
    assert parse_nested_parens('(((())))') == [4]
    assert parse_nested_parens('(()) ()(())') == [2, 2]
test_parse_nested_parens()


[1]
